In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
sys.setrecursionlimit(5000)

In [3]:
from utils import ee_utils, date_utils
from utils.ee_utils import back_to_float, back_to_int, export_image_to_asset
from utils.harmonic_regressor import HarmonicRegressor

from vegetation_period_NDVI.data_loading import add_time_data

from typing import List, Tuple

import ee
import geemap

In [ ]:
ee.Initialize(project="thurgau-irrigation")

## Fill gaps in Landsat ET product using Harmonic Regression
### The units are mm/month NOT scaled. The gap filled collection will be scaled

In [ ]:
# cantonal_borders_asset = (
#     "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"
# )

# aoi_feature_collection = ee.FeatureCollection(cantonal_borders_asset)
# aoi_geometry = aoi_feature_collection.geometry()
# aoi_geometry = aoi_geometry.simplify(500)
# aoi_buffered = aoi_geometry.buffer(100)

cantonal_borders_asset = (
    "projects/thurgau-irrigation/assets/Zuerich/Zuerich_bound"
)

aoi_feature_collection = ee.FeatureCollection(cantonal_borders_asset)
aoi_geometry = aoi_feature_collection.geometry()
aoi_geometry = aoi_geometry.simplify(500)
aoi_buffered = aoi_geometry.buffer(100)

In [ ]:
start_date = ee.Date("2018-01-01")
end_date = ee.Date("2023-01-01")

landsat_data = ee.ImageCollection("projects/thurgau-irrigation/assets/ETlandsatmonthly")

landsat_data = landsat_data.filterDate(start_date, end_date).filterBounds(aoi_buffered)

PROJECTION = landsat_data.first().projection()
SCALE = PROJECTION.nominalScale()

# Prepare the bands for the harmonic regression
landsat_data = landsat_data.map(add_time_data)

landsat_data_list = landsat_data.toList(landsat_data.size())

In [ ]:
regressor = HarmonicRegressor(vegetation_index="ET", omega=1, max_harmonic_order=1)

regressor.fit(landsat_data)
landsat_data_gap_filled = regressor.predict(landsat_data)

landsat_data_gap_filled = landsat_data_gap_filled.map(
    lambda img: back_to_int(
        img.select(["fitted"])
        .rename(f"fitted_ET")
        .setDefaultProjection(crs=PROJECTION, scale=SCALE),
        100,
    )
)

landsat_data_gap_filled_list = landsat_data_gap_filled.toList(
    landsat_data_gap_filled.size()
)

# # Print the projection of the first image before and after the gap filling
# print(landsat_data.first().select("ET").projection().getInfo())
# print(landsat_data_gap_filled.first().projection().getInfo())

In [ ]:
# Map = geemap.Map()

# image_first = ee.Image(landsat_data_list.get(18))
# image_after = ee.Image(landsat_data_gap_filled_list.get(18)).divide(100)

# vis_params = {
#     "bands": ["ET"],
#     "min": 0,
#     "max": 100,
#     "palette": ["blue", "yellow", "orange", "red"],
# }

# vis_params_fitted = {
#     "bands": ["fitted_ET"],
#     "min": 0,
#     "max": 100,
#     "palette": ["blue", "yellow", "orange", "red"],
# }

# Map.center_object(image_first, 10)
# Map.addLayer(image_first, vis_params, "ET")
# Map.addLayer(image_after, vis_params_fitted, "ET fitted")

# Map


## Export the gap filled Landsat ET collection. 
### The collection has now been scaled by 100!

In [ ]:
def export_landsat_ET(
    landsat_et_collection: ee.ImageCollection,
    year: str,
    scale: float,
) -> List[ee.batch.Task]:
    """
    Export the Landsat ET data for a given year.

    Args:
        landsat_et_collection (ee.ImageCollection): The Landsat ET data collection.
        year (str): The year for which the data should be exported.
        scale (float): The scale of the exported images.


    Returns:
        List[ee.batch.Task]: A list of export tasks for the downscaled images.
    """
    landsat_et_collection_list = landsat_et_collection.toList(
        landsat_et_collection.size()
    )

    tasks = []
    for i in range(12):
        m = i + 1
        date = ee.Date.fromYMD(int(year), m, 1)
        time_step_name = f"{m:02d}"

        landsat_image = ee.Image(landsat_et_collection_list.get(i))

        # Change the date to the first day of the month
        landsat_image = landsat_image.set("system:time_start", date.millis())

        task_name = f"Landsat_ET_gap_filled_{year}-{time_step_name}"
        asset_id = f"projects/thurgau-irrigation/assets/Thurgau/Landsat_ET_gap_filled_2018-2022/{task_name}"

        task = export_image_to_asset(
            landsat_image, asset_id, task_name, year, aoi_buffered
        )
        tasks.append(task)

    return tasks


years = range(2018, 2023)

for year in years:

    landsat_data_gap_filled_year = landsat_data_gap_filled.filterDate(
        ee.Date(f"{year}-01-01"), ee.Date(f"{year}-12-31")
    )

    tasks = export_landsat_ET(landsat_data_gap_filled_year, year, 30)

## Verify that it all worked well. Also comparing the Landsat product to the WaPOR product

In [ ]:
landsat_gap_filled_check = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Thurgau/Landsat_ET_gap_filled_2018-2022"
).map(lambda img: back_to_float(img, 100))

years_to_process = range(2018, 2023)

wapor_monthly_asset = "projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_10m_monthly"

WaPOR_et_collections = ee_utils.merge_collections(
    years=years_to_process, asset_name=wapor_monthly_asset
).map(lambda img: back_to_float(img, 100))

In [ ]:
# Filter both collection for 2019
WaPOR_et_2019 = WaPOR_et_collections.filterDate(
    ee.Date("2019-01-01"), ee.Date("2019-12-31")
)

landsat_et_2019 = landsat_gap_filled_check.filterDate(
    ee.Date("2019-01-01"), ee.Date("2019-12-31")
)

landsat_data_raw = ee.ImageCollection("projects/thurgau-irrigation/assets/ETlandsatmonthly")

landsat_data_raw_2019 = landsat_data_raw.filterDate(ee.Date("2019-01-01"), ee.Date("2019-12-31")).filterBounds(aoi_buffered)

In [ ]:
landsat_gap_filled_check_list = landsat_et_2019.toList(
    landsat_et_2019.size()
)

WaPOR_et_collections_list = WaPOR_et_2019.toList(WaPOR_et_2019.size())

landsat_data_raw_2019_list = landsat_data_raw_2019.toList(landsat_data_raw_2019.size())

# Check that the size for both is 12
print(landsat_et_2019.size().getInfo())
print(WaPOR_et_2019.size().getInfo())
print(landsat_data_raw_2019.size().getInfo())

In [ ]:
# date_utils.print_collection_dates(landsat_gap_filled_check)
# date_utils.print_collection_dates(WaPOR_et_collections)

In [ ]:
Map = geemap.Map()

landsat = ee.Image(landsat_gap_filled_check_list.get(8))
WaPOR = ee.Image(WaPOR_et_collections_list.get(8))
landsat_raw = ee.Image(landsat_data_raw_2019_list.get(8))

landsat_params = {
    "bands": ["fitted_ET"],
    "min": 0,
    "max": 200,
    "palette": ["blue", "green", "yellow", "orange", "red"],
}

WaPOR_params = {
    "bands": ["downscaled"],
    "min": 0,
    "max": 200,
    "palette": ["blue", "green", "yellow", "orange", "red"],
}

landsat_raw_params = {
    "bands": ["ET"],
    "min": 0,
    "max": 200,
    "palette": ["blue", "green", "yellow", "orange", "red"],
}

Map.center_object(aoi_buffered, 10)
Map.addLayer(landsat, landsat_params, "Landsat ET")
Map.addLayer(WaPOR, WaPOR_params, "WaPOR ET")
Map.addLayer(landsat_raw, landsat_raw_params, "Landsat ET raw")

Map

# TODO: VERIFY THAT MONTHLY LANDSAT IS GOOD NOW. ALSO LOOK AT THE VALIDATION PLOTS

### Thoughts: looks a bit high to me